### IMPORT 

In [39]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

In [40]:
# 

# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = tf.keras.utils.get_file(origin=dataset_url, fname='flower_photos', untar=True)
# print(data_dir)
# data_dir = pathlib.Path(data_dir)
# data_dir

### GET Dir

In [41]:
data_dir = pathlib.Path("plant")
data_dir

WindowsPath('plant')

### SET SIZE

In [42]:
batch_size = 32
img_height = 194
img_width = 194

### SCALE

In [43]:
list_dir = os.listdir(data_dir)
for dir in list_dir:
    path_dir = pathlib.Path("plant/" + dir)
    files = os.listdir(path_dir)
    for file in files:
        file_path = "plant/" + dir + "/" + file
        if os.path.isfile(file_path):
            try:
                im = PIL.Image.open(file_path)
                f, e = os.path.splitext(file_path)
                imResize = im.resize((194,194), PIL.Image.ANTIALIAS)
                imResize.save(f+e, quality=90)
            except:
                continue

## [TODO] : Remove corrupted picture

C:\Users\abeme\AppData\Local\Temp\ipykernel_8340\1284190516.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((194,194), PIL.Image.ANTIALIAS)


### SPLIT DATA

In [48]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed = 1,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed = 1,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 4380 files belonging to 9 classes.
Using 3504 files for training.
Found 4380 files belonging to 9 classes.
Using 876 files for validation.


### NORMALIZE

In [52]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [54]:
normalized_ds = train_ds.map(lambda x,y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input is empty.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext]

In [56]:
num_classes = 9

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [57]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


### TRAIN and FIT

In [58]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=6
)

Epoch 1/6


InvalidArgumentError: Graph execution error:

Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_3363]